In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.stats import dirichlet, norm, uniform, bernoulli, beta
import timeit
from itertools import permutations 
import itertools

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
np.random.seed(seed=1)

Consider SNP data of $N$ individual at $L$ loci, denote by $X = (x_{ij})_{i=\overline{1, N}, l = \overline{1, L}}$. We now implement the population structure algorithm presented in [1] using HDP.

For $=\overline{1, N}, l = \overline{1, L}$, denote $z_{il}$ the population ancestry at locus $l$ of individual $l$, and $Q_i = (q_{i1}, \dots, q_{ik})$ the proportions of individual $i$ belongs to each of $k$ population ancestry. We also consider linkage disquilibrium by including the random variables $s_{il} \in \{0,1 \}$, where $s_{il} = 1$ means locus $l-1$ and $l$ are in the same chunk ($=0$ if not). The probability transition model is

$$z_{i1} \sim \text{discrete}(Q_i), $$

$$s_{i, l+1} \overset{iid}{\sim} \text{Bernoulli}(e^{-rd_l}), \quad l= 1, \dots, L-1, $$

$$z_{i, l+1}|s_{i, l+1} z_{il} \begin{cases} = z_{il} & \text{if } s_{i, l+1} = 1,\\
\sim \text{Bernoulli}(e^{-rd_l}) & \text{if } s_{i, l+1} = 0,
\end{cases}$$
where $d_l$ is the genetic distance between locus $l-1$ and $l$.

We assume that there is Hardy-Weinberg equilibrium within each population. Denote $\theta_k = (\theta_{kla})_{l=\overline{1, L}, a = 0, 1}$ be the probability of allele at locus $l$ of population $k$ to be 0/1 (because we only consider SNP case, in microsattlite, $a$ can be $\geq 2$). We have

$$P(x_{il} = a | z_{il}=k) = \theta_{kla}, \quad \forall \, i = \overline{1, N}, l = \overline{1, L}, a = 0,1.$$


Finally, we equip prior for $Q_i$ using Hierachical Dirichlet Process. 

$$Q_i | Q_0 \overset{iid}{\sim} \text{DP}(\alpha Q_0), $$

$$Q_0 \sim \text{DP}(\alpha_0 H). $$

In [15]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

In [25]:
## functions to help sample latent variables
def count_n_m(z, s):
    """
    Count n[i, k], i = 0,..., N-1, k = 0,..., K-1 based on z, s
    """
    n = np.zeros((N, K))
    m = np.zeros((N, K))
    for i in range(N):
        for k in range(K):
            n[i, k] = np.sum((z[i, l] == k) & (s[i, l] == 0) for l in range(L)) 
            if n[i,k] > 0:
                m[i,k] = np.sum([bernoulli(α * q[i,k] / (α * q[i,k] + j)).rvs() for j in range(n[i,k])])
            
    return n, m


def Q0(α0, m, K, KMAX):
    n0 = [np.sum(m[:, k]) for k in range(K)]
    q0 = dirichlet(n0.append(α0)).rvs()[0]             ## q_{01},..., q_{0K}, w_0
    q0tmp = np.zeros(KMAX)
    # sample from Q0'
    for j in range(KMAX):
        l0 = 1    # stick length
        b0 = beta(1, α0).rvs()
        l0new = l0 * b0
        q0tmp[j] = l0new
        l0 = l0 * (1-b0)
    q0tmp = q0tmp * q0[K]  ## mass of the stick breaking G'
    q0star = l0 * q0[K]    ## left-over mass

    return np.append(q0[:-1], q0tmp, q0star)

def LD(r, d):
    """
    Linkage disequilibrium paramter  
    Input: r: a postive number 
    d = d[l] (np array) for l in range(L-1) distance between locus l and l+1 (if dont know, just let all = 1)
    """
    ld = np.exp(- r * d)
    return ld    

In [32]:
## posterior sample z and s of instance i
def FFBS(popi, qi, ld, θ, X):
    """
    popi (length Ki): set of populations having prob > Ci
    qi (length Ki): their probability
    return proposed zi and si.

    Foward filtering:
    M[b,k,l] = P(x, s[i, l] = b, z[i, l] = k| q, θ) for b = 0, 1; k = 0,..., Ki-1; i = 0,..., N-1; l = 0,..., L-1
    Md[k, l] = p(x, z[i, l] = k | q, θ) for k = 0,..., Ki-1; i = 0,..., N-1; l = 0,..., L-1
    Mdd[l] = p(x | q, θ) for l = 0,..., L-1
    
    then backward sampling z_i and s_i:
    Q(z[i, L] = k) α Md[k, i, L]
    Q(s[i, L] = b | z[i, L] = k) α M[b, k, i, L]
    Q(z[i, l] = k | s[i, l+1] = 1, z[i, l+1] = k') α 1(k = k')
    Q(z[i, l] = k | s[i, l+1] = 0, z[i, l+1] = k') α Md[k, i, l]
    """
    Ki = len(popi)
    M = np.zeros((2, Ki, L))
    Md = np.zeros((Ki, L))
    Mdd = np.zeros(L)
    
    ## Forward filtering
    Md[:, 0] = θ[popi, 0, X[i, 0]] * qi
    Mdd[0] = np.sum(Md[:, 0])
    
    for l in range(1, L):
        M[1, :, l] = θ[popi, l, X[i, l]] * Md[:, l-1] * ld[l-1]
        M[0, :, l] = θ[popi, l, X[i, l]] * Mdd[:, l-1] * qi * (1-ld[l-1])
        Md[:, l] = M[1, :, l] + M[0, :, l]
        Mdd[l] = np.sum(Md[:, l])
        
    ## Backward sampling
    zi = np.zeros(L)
    si = np.zeros(L)
    
    zi[L-1] = np.random.choice(popi, p = Md[:, L-1] / np.sum(Md[:, L-1]))
    si[L-1] = np.random.choice(2, p = M[:, zi[L-1], L-1] / np.sum(M[:, zi[L-1], L-1]))
    
    for l in reversed(range(L-1)):
        if si[l+1] == 1:
            zi[l] = zi[l+1]
        else:
            zi[l] = np.random.choice(popi, p = Md[:, l] / np.sum(Md[:, l]))
            
        si[l] = np.random.choice(2, p = M[:, zi[l], l] / np.sum(M[:, zi[l], l]))
    
    return zi, si 
        
def ZS(θ, x, q0, ni, zi_old, si_old, α):
    """
    Sample z_i and s_i This step can be paralllel because Gi are independent given G0
    q0 = Q0(...)
    """
    
    K = len(q0) - 1
    param = np.append(α * q0[:K] + ni, α * q0[K])
    qi = dirichlet(param).rvs()[0]
    q_min_curr = np.min(qi[np.unique(zi_old)])  ## CAREFUL: what if zi_old has a cell > K
    Ci = uniform(0, q_min_curr).rvs()
    popi = [ind for ind in range(K) if qi[ind] > Ci]
    qi_trunc = qi[popi]
    
    zi, si = FFBS(popi, qi_trunc, ld, θ, X)
    
    q_min_prop = np.min(qi[np.unique(zi)])
    if uniform().rvs() > (q_min_curr / q_min_prop):
        zi = zi_old
        si = si_old    
    
    return zi, si

def theta(X, Z, a0, b0):
    θ = np.zeros((K, L, 2))
    for k in range(K):
        for l in range(L):
            apost = a0 + np.sum(np.logical_and([X[:, l] == 0], [Z[:, l] == k]))
            bpost = b0 + np.sum(np.logical_and([X[:, l] == 1], [Z[:, l] == k]))
            θ[k, l, 0], θ[k, l, 1] = dirichlet([apost, bpost]).rvs()[0]
    return θ